Para rodar o notebook testes_ocr.ipynb
 completamente, você precisará preparar o ambiente e criar contas em alguns serviços, pois ele utiliza tanto ferramentas locais quanto APIs na nuvem.

Aqui está o checklist do que você precisa fazer:

1. Instalações no Sistema (Mac)
O notebook usa o Tesseract (para OCR básico) e o Poppler (para manipular PDFs). Eles não são instalados via Python, precisam ser instalados no sistema. Abra seu terminal e rode:

```
bash
brew install tesseract poppler
```

2. Dependências Python
No próprio notebook, a primeira célula de código instala as bibliotecas necessárias. Certifique-se de rodá-la:

```
python
!pip install opencv-python pytesseract pdf2image easyocr pdfplumber matplotlib numpy
!pip install landingai docling llama-parse unstructured
```

3. Contas e Chaves de API (Necessário Cadastro)

Algumas bibliotecas do notebook são serviços pagos ou que exigem autenticação na nuvem. Você precisará criar conta para obter as chaves:

LlamaParse (LlamaIndex)
- Para que serve: Extração inteligente de PDFs para RAG.
- Onde criar conta: Vá ao site da LlamaCloud e faça login (geralmente via GitHub/Google).
- O que fazer: Gere uma LLAMA_CLOUD_API_KEY e coloque-a na célula correspondente do notebook.

LandingAI (Vision Agent)
- Para que serve: Agente de visão computacional que aceita comandos em texto natural.
- Onde criar conta: Vá ao site da LandingAI (ou LandingLens).
O que fazer: Obtenha a LANDINGAI_API_KEY. (Nota: verifique se há free tier disponível, pois é um serviço comercial).

4. Arquivos de Teste

O notebook faz referência a arquivos locais que não parecem existir na pasta ainda:

- exemplo_nota_fiscal.pdf
- exemplo_manuscrito.jpg
- exemplo_tabela.pdf

Você precisará colocar arquivos reais com esses nomes na mesma pasta do notebook ou alterar o código para apontar para seus próprios arquivos de teste.

Resumo

Se você quiser rodar apenas o básico (Tesseract, EasyOCR, Docling, Unstructured local), só precisa das instalações no passo 1 e 2. Se quiser testar as ferramentas avançadas de IA (LlamaParse, LandingAI), precisará dos cadastros no passo 3.

# Testes de Técnicas de OCR para Documentos Diversos

Este notebook tem como objetivo testar e comparar diferentes bibliotecas e técnicas de OCR (Optical Character Recognition) para leitura de:
1. Notas fiscais (Documentos estruturados/semi-estruturados)
2. Tabelas em PDFs
3. Imagens com texto escrito à mão

## Bibliotecas Utilizadas:
- **Tesseract (pytesseract)**: Padrão da indústria, bom para textos digitalizados limpos.
- **EasyOCR**: Baseado em Deep Learning, costuma ser mais robusto para texto em cenários naturais e escrita manual.
- **pdfplumber**: Excelente para extração direta de dados e tabelas de PDFs nativos (não escaneados).
- **OpenCV**: Para pré-processamento de imagens (binarização, redução de ruído).
- **LandingAI DPT-2**: Document Pre-trained Transformer para extração inteligente.
- **IBM Docling**: Solução completa de parsing de documentos.
- **LlamaParse**: Parser poderoso da LlamaIndex focado em RAG.
- **Unstructured.io**: Toolkit flexível para ingestão de dados não estruturados.

## 1. Instalação e Configuração
**Nota:** Para o `pytesseract` funcionar, você precisa ter o binário do Tesseract instalado no seu sistema:
- **Mac**: `brew install tesseract poppler`
- **Linux**: `sudo apt install tesseract-ocr poppler-utils`
- **Windows**: Baixar o instalador oficial.

In [ ]:
# Instalação das dependências Python
!pip install opencv-python pytesseract pdf2image easyocr pdfplumber matplotlib numpy
!pip install landingai docling llama-parse unstructured

In [ ]:
import cv2
import pytesseract
import easyocr
import pdfplumber
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import numpy as np
import os

# Configuração para exibir imagens no notebook
def mostrar_imagem(img, titulo="Imagem"):
    plt.figure(figsize=(10, 10))
    if len(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img, cmap='gray')
    plt.title(titulo)
    plt.axis('off')
    plt.show()

## 2. Preparação dos Dados
Coloque seus arquivos de teste na pasta local ou defina os caminhos abaixo.

In [ ]:
# Defina os caminhos dos seus arquivos de teste aqui
arquivo_pdf_nota = "exemplo_nota_fiscal.pdf"  # Substitua pelo seu arquivo
arquivo_imagem_mao = "exemplo_manuscrito.jpg" # Substitua pelo seu arquivo
arquivo_pdf_tabela = "exemplo_tabela.pdf"     # Substitua pelo seu arquivo

## 3. OCR com Tesseract (Generalista)

In [ ]:
def ocr_tesseract(caminho_imagem):
    img = cv2.imread(caminho_imagem)
    
    # Pré-processamento simples: converter para escala de cinza
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Binarização (Threshold) pode ajudar em documentos ruidosos
    # _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    
    mostrar_imagem(img, "Imagem Original")
    
    # OCR
    # config='--psm 6' assume um bloco único de texto uniforme
    texto = pytesseract.image_to_string(gray, lang='por') 
    return texto

# Exemplo de uso (descomente se tiver o arquivo):
# print("--- Resultado Tesseract ---")
# print(ocr_tesseract(arquivo_imagem_mao))

## 4. OCR com EasyOCR (Manuscrito e Deep Learning)
O EasyOCR costuma ser melhor para textos artísticos, manuscritos ou em fotos naturais.

In [ ]:
reader = easyocr.Reader(['pt', 'en'], gpu=False) # Defina gpu=True se tiver CUDA

def ocr_easyocr(caminho_imagem):
    img = cv2.imread(caminho_imagem)
    mostrar_imagem(img, "Entrada EasyOCR")
    
    result = reader.readtext(caminho_imagem)
    
    texto_completo = ""
    for (bbox, text, prob) in result:
        if prob > 0.5: # Filtrar leituras com baixa confiança
            print(f"Detectado: {text} (Confiança: {prob:.2f})")
            texto_completo += text + " "
            
            # Desenhando bbox na imagem para visualização
            (tl, tr, br, bl) = bbox
            tl = (int(tl[0]), int(tl[1]))
            br = (int(br[0]), int(br[1]))
            cv2.rectangle(img, tl, br, (0, 255, 0), 2)
            
    mostrar_imagem(img, "Detectado EasyOCR")
    return texto_completo

# Exemplo de uso:
# print(ocr_easyocr(arquivo_imagem_mao))

## 5. Extração de Tabelas de PDFs (PdfPlumber)
Para notas fiscais e relatórios que são PDFs digitais (text-based), não use OCR de imagem. Use extração direta.

In [ ]:
def extrair_tabelas_pdf(caminho_pdf):
    with pdfplumber.open(caminho_pdf) as pdf:
        for i, page in enumerate(pdf.pages):
            print(f"--- Página {i+1} ---")
            
            # Extrair texto puro
            texto = page.extract_text()
            print("Conteúdo textual:")
            print(texto[:200] + "...") # Preview
            
            # Extrair tabelas
            tabelas = page.extract_tables()
            for j, tabela in enumerate(tabelas):
                print(f"\nTabela {j+1}:")
                for linha in tabela:
                    print(linha)
                    
# Exemplo de uso:
# extrair_tabelas_pdf(arquivo_pdf_tabela)

## 6. Processamento de Imagens de Notas Fiscais (OCR + Regex)
Muitas vezes precisamos localizar campos específicos (CNPJ, Data, Valor) após o OCR.

In [ ]:
import re

def buscar_info_nota(texto):
    # Exemplos de Regex comuns em notas brasileiras
    regex_cnpj = r"\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}"
    regex_data = r"\d{2}/\d{2}/\d{4}"
    regex_valor = r"R\$ ?[\d\.,]+"
    
    cnpjs = re.findall(regex_cnpj, texto)
    datas = re.findall(regex_data, texto)
    valores = re.findall(regex_valor, texto)
    
    return {
        "cnpjs": cnpjs,
        "datas": datas,
        "valores": valores
    }

# Teste integrado
# texto_extraido = ocr_tesseract(arquivo_pdf_nota_convertido_para_img)
# info = buscar_info_nota(texto_extraido)
# print(info)

## 7. LandingAI DPT-2 (Vision Agent)
O DPT-2 é focado em transformar documentos visuais em dados estruturados usando transformers pré-treinados.
**Nota**: Necessita API Key.

In [ ]:
# !pip install landingai
import os

# Defina sua chave de API aqui
# os.environ["LANDINGAI_API_KEY"] = "sua-chave-aqui"

try:
    from landingai.pipeline.image_source import ImageSource
    # Exemplo genérico de uso da lib (ajuste conforme a documentação mais recente do Vision Agent)
    # O Vision Agent permite comandos em linguagem natural para processar a imagem
    
    def processar_landingai(caminho_imagem):
        if "LANDINGAI_API_KEY" not in os.environ:
            print("Erro: LANDINGAI_API_KEY não definida.")
            return
            
        print("Iniciando LandingAI...")
        # Exemplo hipotético de uso do agente
        # from landingai.agent import VisionAgent
        # agent = VisionAgent()
        # resposta = agent.run("Extraia o valor total e a data desta nota fiscal.", image_path=caminho_imagem)
        # print(resposta)
        
except ImportError:
    print("Biblioteca landingai não instalada ou erro na importação.")

## 8. IBM Docling
Docling é uma biblioteca poderosa da IBM para conversão de documentos complexos (PDFs com tabelas, fórmulas, etc) em formatos estruturados como Markdown ou JSON.

In [ ]:
# !pip install docling

try:
    from docling.document_converter import DocumentConverter

    def processar_docling(caminho_fonte):
        print(f"Processando {caminho_fonte} com Docling...")
        converter = DocumentConverter()
        result = converter.convert(caminho_fonte)
        
        # Exportar para Markdown
        md_output = result.document.export_to_markdown()
        print("--- Resultado Markdown (Docling) ---")
        print(md_output[:500] + "...") # Preview
        return md_output

    # Exemplo de uso:
    # processar_docling(arquivo_pdf_nota)

except ImportError:
    print("Biblioteca docling não instalada.")
except Exception as e:
    print(f"Erro ao executar Docling: {e}")

## 9. LlamaParse (LlamaIndex)
Projetado especificamente para aplicações RAG (Retrieval-Augmented Generation), o LlamaParse é excelente em transformar documentos complexos em texto limpo.
**Nota**: Requer API Key da LlamaCloud.

In [ ]:
# !pip install llama-parse
import os

# os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."

try:
    import nest_asyncio
    nest_asyncio.apply() # Necessário para rodar loops de evento em notebooks
    from llama_parse import LlamaParse

    def processar_llamaparse(caminho_pdf):
        if "LLAMA_CLOUD_API_KEY" not in os.environ:
            print("Erro: LLAMA_CLOUD_API_KEY não definida.")
            return

        print(f"Processando {caminho_pdf} com LlamaParse...")
        parser = LlamaParse(
            result_type="markdown",  # "markdown" ou "text"
            verbose=True,
            language="pt" # Opcional
        )
        
        # O LlamaParse pode carregar o arquivo e parsear
        documents = parser.load_data(caminho_pdf)
        
        for doc in documents:
            print("--- Página/Trecho ---")
            print(doc.text[:500] + "...")
            
    # Exemplo de uso
    # processar_llamaparse(arquivo_pdf_nota)

except ImportError:
    print("Biblioteca llama-parse não instalada.")
except Exception as e:
    print(f"Erro no LlamaParse: {e}")

## 10. Unstructured.io
Ferramenta versátil para ingestão ("partitioning") de diversos formatos (PDF, DOCX, HTML, Imagens). Pode rodar local (dependências de sistema) ou via API.

In [ ]:
# !pip install unstructured
# Para suporte a PDF local, é necessário instalar dependências do sistema como poppler e tesseract

try:
    from unstructured.partition.pdf import partition_pdf
    from unstructured.partition.image import partition_image

    def processar_unstructured_pdf(caminho_pdf):
        print(f"Processando {caminho_pdf} com Unstructured...")
        
        # partition_pdf tenta extrair texto, tabelas, etc.
        # strategy="hi_res" usa OCR e análise visual (mais lento, mais preciso)
        # strategy="fast" extrai texto diretamente (somente PDFs nativos)
        elements = partition_pdf(
            filename=caminho_pdf,
            strategy="hi_res", # Mude para "fast" para teste rápido
            infer_table_structure=True
        )
        
        print(f"Total de elementos extraídos: {len(elements)}")
        for el in elements[:5]:
            print(f"[{el.category}]: {el.text}")
            
    # Exemplo de uso
    # processar_unstructured_pdf(arquivo_pdf_nota)

except ImportError:
    print("Biblioteca unstructured não instalada.")
except Exception as e:
    print(f"Erro no Unstructured: {e}. Verifique se poppler/tesseract estão instalados.")